In [1]:
# !pip install -q -U google-generativeai
import google.generativeai as genai
import pandas as pd
import numpy as np
import os
import time
from getpass import getpass

c:\Users\Rafael\Desktop\Prog\TCC\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
os.environ['GOOGLE_API_KEY'] = getpass("Por favor, cole sua chave de API aqui: ")
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

safety_settings = [
    {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'threshold': 'BLOCK_NONE'},
    {'category': 'HARM_CATEGORY_HATE_SPEECH', 'threshold': 'BLOCK_NONE'},
    {'category': 'HARM_CATEGORY_HARASSMENT', 'threshold': 'BLOCK_NONE'},
    {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'threshold': 'BLOCK_NONE'}
]

model = genai.GenerativeModel('gemini-1.5-flash', safety_settings=safety_settings)

'Please provide me with a prompt! I need something to work with in order to give you a helpful response. \n\nFor example, you could ask me to:\n\n* **Write a story:** "Write a short story about a cat who becomes a detective."\n* **Generate code:** "Write a Python function that checks if a number is prime."\n* **Translate text:** "Translate \'Hello, how are you?\' into Spanish."\n* **Summarize a text:** "Summarize the main points of the article about climate change."\n* **Answer a question:** "What is the capital of France?"\n\nLet me know what you want me to do! \n'

In [9]:
path = "../data/"
df = pd.read_excel(path + "americanas_products.xlsx")
df.head()

,Class,Product name,Id
0,caminhao sonoro,Caminhão Sonoro - Brinquedo Educativo de Madei...,2471056238
1,caminhao sonoro,Caminhão Sonoro Em Madeira - Carimbras,2783873157
2,caminhao sonoro,Fecho Cinto Segurança Dianteiro Caminhão Com S...,1625230957
3,caminhao sonoro,Dni0526 Alarme Sinalizador Sonoro E Visual par...,2469110493
4,caminhao sonoro,Caminhão Sonoro - Brinquedo Educativo de Madei...,29704506


In [10]:
df["LLM classification"] = np.nan
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 490 entries, 0 to 489
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Class               490 non-null    object 
 1   Product name        490 non-null    object 
 2   Id                  490 non-null    int64  
 3   LLM classification  0 non-null      float64
dtypes: float64(1), int64(1), object(2)
memory usage: 15.4+ KB


In [11]:
def get_classification(product, classes):

    prompt = f"""
    Contexto: Você receberá o título de um produto proveniente de um site de marketplace.
    Objetivo: Classifique o produto em uma das seguintes categorias: {classes}.
    Formatação: Escreva apenas a classificação do produto em uma linha.
    
    O produto é: {product}
    """
    response = model.generate_content(prompt)
    print(f"Produto: {product}\nResposta: {response.text}")
    time.sleep(4)
    return response.text

In [6]:
classes = df["Class"].unique()

for index, row in df.iterrows():
    df.at[index, "LLM classification"] = get_classification(row["Product name"], classes)
    df.to_excel(path + "americanas_products_classified.xlsx", index=False)

Produto: Caminhão Sonoro - Brinquedo Educativo de Madeira - Carimbras
Resposta: caminhao sonoro 



KeyboardInterrupt: 

In [12]:
# Remover espaços da coluna "LLM classification", e colocar tudo em minusculo
df["LLM classification"] = df["LLM classification"].str.strip().str.lower()

AttributeError: Can only use .str accessor with string values!

In [ ]:
#print lines from df where LLM classification is not different from the original classification
dfErros = df[df["Class"] != df["LLM classification"]]
dfErros = dfErros.drop(columns=["Product name", "Id"])
df["LLM classification"].unique()

array(['caminhao sonoro', 'peruca', 'rodo', 'biscoito wafer',
       'rolamento carro', 'massa', 'papel camurca', 'progressiva',
       'puxador porta', 'amaciante calo', 'bule', 'vibrador',
       'oleo reparador', 'papel sulfite', 'porta papel higienico',
       'cortador', 'botao acionamento caixa', 'ampola capilar',
       'espelho passaro', 'brinquedo relogio', 'pino macho',
       'adaptador tomada', 'mascara cabelo', 'tira mancha pet',
       'painel luz led', 'tasso', 'moletom', 'gel fixador',
       'oleo essencial', 'lente lanterna carro', 'tiara', 'ponta solda',
       'suporte smartphone', 'brinquedo bola corda pet', 'dock station',
       'controle disparador', 'suplemento alimentar infantil',
       'armario aereo', 'sapatilha', 'fechadura porta', 'fotocondutor',
       'limpador oral', 'carregador usb', 'bandana pet', 'caixa acustica',
       'touca', 'bomba vacuo', 'moldura quadro', 'teclado musical',
       'patch'], dtype=object)

In [ ]:
# get classification report from df
from sklearn.metrics import classification_report
print(classification_report(df["Class"], df["LLM classification"]))

                               precision    recall  f1-score   support

             adaptador tomada       0.83      1.00      0.91        10
               amaciante calo       1.00      1.00      1.00        10
               ampola capilar       1.00      1.00      1.00         9
                armario aereo       1.00      1.00      1.00        10
                  bandana pet       1.00      1.00      1.00        10
               biscoito wafer       1.00      1.00      1.00        10
                  bomba vacuo       1.00      1.00      1.00        10
      botao acionamento caixa       1.00      1.00      1.00        10
     brinquedo bola corda pet       1.00      1.00      1.00        10
            brinquedo relogio       0.91      1.00      0.95        10
                         bule       1.00      1.00      1.00        10
               caixa acustica       1.00      1.00      1.00        10
              caminhao sonoro       0.50      1.00      0.67        10
     

In [ ]:
df['Class'].value_counts()

Class
caminhao sonoro                  10
peruca                           10
rodo                             10
biscoito wafer                   10
massa                            10
papel camurca                    10
amaciante calo                   10
puxador porta                    10
bule                             10
vibrador                         10
cortador                         10
oleo reparador                   10
papel sulfite                    10
porta papel higienico            10
espelho passaro                  10
botao acionamento caixa          10
mascara cabelo                   10
pino macho                       10
caixa acustica                   10
armario aereo                    10
suporte smartphone               10
painel luz led                   10
tasso                            10
brinquedo relogio                10
moletom                          10
gel fixador                      10
ponta solda                      10
oleo essencial        

In [ ]:
# df['LLM classification'] where llm classification == "caminhao sonoro"

df[df['LLM classification'] == "caminhao sonoro"]

,Class,Product name,Id,LLM classification
0,caminhao sonoro,Caminhão Sonoro - Brinquedo Educativo de Madei...,2471056238,caminhao sonoro
1,caminhao sonoro,Caminhão Sonoro Em Madeira - Carimbras,2783873157,caminhao sonoro
2,caminhao sonoro,Fecho Cinto Segurança Dianteiro Caminhão Com S...,1625230957,caminhao sonoro
3,caminhao sonoro,Dni0526 Alarme Sinalizador Sonoro E Visual par...,2469110493,caminhao sonoro
4,caminhao sonoro,Caminhão Sonoro - Brinquedo Educativo de Madei...,29704506,caminhao sonoro
5,caminhao sonoro,Caminhão Sonoro - Brinquedo Educativo de Madei...,3899898031,caminhao sonoro
6,caminhao sonoro,Dni0526 Alarme Sinalizador Sonoro E Visual par...,4998329853,caminhao sonoro
7,caminhao sonoro,Caminhão Sonoro Em Madeira - Carimbras,5895419119,caminhao sonoro
8,caminhao sonoro,Carrinhos de brinquedo coloridos com caminhão ...,7432785681,caminhao sonoro
9,caminhao sonoro,"ArtCreativity Light Up Dump Truck Toy, Brinque...",7468944015,caminhao sonoro


In [13]:
product = "Magnazia Sabor Lima-Limão Suspensão Oral 240ml"

prompt = f"""
Contexto: Você receberá o título de um produto proveniente de um site de marketplace.
Objetivo: Classifique o produto em uma das seguintes categorias: {classes}.
Formatação: Escreva apenas a classificação do produto em uma linha.

O produto é: {product}
"""
response = model.generate_content(prompt)
print(f"Produto: {product}\nResposta: {response.text}")

Produto: Magnazia Sabor Lima-Limão Suspensão Oral 240ml
Resposta: suplemento alimentar infantil 



In [14]:
print(prompt)


Contexto: Você receberá o título de um produto proveniente de um site de marketplace.
Objetivo: Classifique o produto em uma das seguintes categorias: ['caminhao sonoro' 'peruca' 'rodo' 'biscoito wafer' 'rolamento carro'
 'massa' 'papel camurca' 'puxador porta' 'amaciante calo' 'bule'
 'vibrador' 'oleo reparador' 'papel sulfite' 'porta papel higienico'
 'cortador' 'botao acionamento caixa' 'ampola capilar' 'espelho passaro'
 'pino macho' 'mascara cabelo' 'suporte smartphone' 'tira mancha pet'
 'painel luz led' 'brinquedo relogio' 'tasso' 'moletom' 'gel fixador'
 'oleo essencial' 'lente lanterna carro' 'tiara' 'ponta solda'
 'brinquedo bola corda pet' 'dock station' 'controle disparador'
 'suplemento alimentar infantil' 'armario aereo' 'sapatilha'
 'fechadura porta' 'fotocondutor' 'limpador oral' 'carregador usb'
 'bandana pet' 'caixa acustica' 'adaptador tomada' 'touca' 'bomba vacuo'
 'moldura quadro' 'teclado musical' 'progressiva' 'patch'].
Formatação: Escreva apenas a classificação

In [19]:
print('suplemento alimentar' in classes)

False
